In [1]:
#!/usr/bin/env python3
"""
Read SO-CI output from Molpro.
Assign omegas, including parity. 
Also show composition of a level, or distribution of a term.
C2v symmetry (4 irreps) is assumed. 
Includes dipole moments
This version uses both elimination and trig strategies
KKI 6/6/2023
"""
import re, sys, copy, glob, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
sys.path.insert(0, '../atomic_SOC')
import molpro_subs as mpr
import chem_subs as chem

pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 500)

In [2]:
#fsoci = 'ac5z_hybB_r2p2444_lz.pro'  # works
fsoci = 'ac5z_soc2011_r1p5987_21S15T_lz.pro'  # even case, works
#fsoci = '../UMemphis/mgcl-equil-karl-block.pro'  # works
#fsoci = '../UMemphis/ch.pro'  # works
#fsoci = '../UMemphis/mgcl_small4.pro'  # works
print('Will read SO-CI info from file: {:s}'.format(fsoci))

Will read SO-CI info from file: ac5z_soc2011_r1p5987_21S15T_lz.pro


In [3]:
# Read CASSCF
PG = mpr.read_point_group(fsoci)
print('Point group is', PG)
crd, lineno_crd = mpr.read_coordinates(fsoci, linenum=True)
if isinstance(lineno_crd, list):
    # take last geometry
    crd = crd[-1]
    lineno_crd = lineno_crd[-1]
# get diatomic bond length
G = chem.Geometry(crd, intype='DataFrame', units='bohr')
G.toAngstrom()
R = np.round(G.distance(0, 1), 6)  # round the bond length to 6 digits
print('Bond length = {:.4f}'.format(R))
caslist, lineno_cas = mpr.readMULTI(fsoci, PG=PG, linenum=True)
CAS = caslist[-1]   # assume the last CASSCF to be the relevant one

oldcas = CAS.results.copy()
for Spin in sorted(set(oldcas.Spin)):
    nspin = len(oldcas[oldcas.Spin == Spin])
    print('{:d} {:s}s'.format(nspin, Spin))
print('Active space = {:d}/{:d}'.format(CAS.nactel(), CAS.nactorb()))
CAS.results = mpr.relabel_CAS_by_energy(CAS.results)
# Note any changes in CAS labeling
diflbl = (oldcas.Label != CAS.results.Label).values
if diflbl.any():
    dflbl = oldcas[['Label', 'Term']].copy()
    dflbl['NewLabel'] = CAS.results.Label
    print('Some CAS labels changed:')
    display(dflbl)
# check for dynamical weights
rx_dynw = re.compile('[~!]*dynw,(\d+)')
dynw = 0
with open(fsoci, 'r') as F:
    for line in F:
        m = rx_dynw.search(line)
        if m:
            dynw = int(m.group(1))
            print('Dynamical weighting with dynw = {:d}'.format(dynw))
if not dynw:
    print('Uniform weighting')

Point group is C2v
Bond length = 1.5987
21 Singlets
15 Triplets
Active space = 20/11
Dynamical weighting with dynw = 8
Dynamical weighting with dynw = 8


In [4]:
# count the terms that are included in the calculation
from collections import Counter
for spin, grp in CAS.results.groupby('Spin'):
    print(spin)
    print(Counter(grp.Term.tolist()))
#CAS.results

Singlet
Counter({'1Δ': 6, '1Π': 6, '1Σ+': 5, '1Γ': 2, '1Φ': 2})
Triplet
Counter({'3Π': 6, '3Δ': 4, '3Φ': 2, '3Σ-': 2, '3Σ+': 1})


In [5]:
casdf = CAS.results[['Irrep', 'Label', 'Energy', 'Term']].copy()
casdf['S'] = [chem.spinname(x)-1 for x in CAS.results.Spin]
casdf['Lz'] = np.round(np.sqrt(np.abs(CAS.results.LzLz)), 0).astype(int)
spin = sorted(set(casdf.S))
irreps = sorted(set(casdf.Irrep))
lzvals = sorted(set(casdf.Lz))

In [6]:
casdf['ecm'] = np.round((casdf.Energy - casdf.Energy.min()) * chem.AU2CM, 0)
casdf.sort_values('Energy')

,Irrep,Label,Energy,Term,S,Lz,ecm
32,4,1.4,-119.078953,3Δ,2,2,0.0
21,1,1.1,-119.078953,3Δ,2,2,0.0
0,1,1.1,-119.075073,1Σ+,0,0,852.0
17,4,1.4,-119.072435,1Δ,0,2,1430.0
1,1,2.1,-119.072435,1Δ,0,2,1430.0
24,2,1.2,-119.070139,3Π,2,1,1935.0
28,3,1.3,-119.070139,3Π,2,1,1935.0
22,1,2.1,-119.063229,3Σ+,2,0,3451.0
9,2,1.2,-119.049726,1Π,0,1,6415.0
13,3,1.3,-119.049726,1Π,0,1,6415.0


In [7]:
# check for proper pairing of degenerate states
broken = False
for grp, dfg in casdf.groupby(['S', 'Lz']):
    if grp[1] == 0:
        # ignore Sigma states
        continue
    # number in each irrep should be equal
    grpi = dfg.groupby('Irrep')
    lens = grpi.size().values
    if lens[0] != lens[1]:
        print('Broken pair somewhere')
        display(dfg.sort_values('Energy'))
        broken = True
if not broken:
    print('All CASSCF state pairs are closed')

All CASSCF state pairs are closed


In [8]:
# check for non-integer Lz values
cruft = casdf.Lz - np.round(casdf.Lz, 0)
if cruft.any():
    print('*** There are non-integer values of Lz')
else:
    print('Lz values look OK')

Lz values look OK


In [9]:
# order the CASSCF states by energy within each (S, irrep) group
newdf = pd.DataFrame(columns=casdf.columns)
for lbl, dg in casdf.groupby(['S', 'Irrep']):
    dt = dg.copy().sort_values('Energy').reset_index(drop=True)
    #display(dt)
    newdf = newdf.append(dt)

In [10]:
def assign_trig(Lzs, irreps):
    # Add the sin/cos descriptor based upon Lz and irrep
    trig = []
    for L, irr in zip(Lzs, irreps):
        if L == 0:
            trig.append('1')
        elif L%2 == 1:
            # odd L
            if irr == 2:
                trig.append('cos')
            elif irr == 3:
                trig.append('sin')
            else:
                trig.append('???')
        else:
            # even L
            if irr == 1:
                trig.append('cos')
            elif irr == 4:
                trig.append('sin')
            else:
                trig.append('???')
    return trig

In [11]:
# read MRCI
cilist, lineno_ci = mpr.readMRCI(fsoci, linenum=True)   # probably many
for m in cilist:
    m.transfer_lz(CAS.results)
mrci = [mpr.MRCIstate(row) for m in cilist for (irow, row) in m.results.iterrows()]
dfci = mpr.combineMRCI(cilist)
# Add the trig column
dfci['trig'] = assign_trig(dfci.Lz, dfci.Irrep)

In [12]:
dfci['ecm'] = np.round( (dfci.Edav - dfci.Edav.min()) * chem.AU2CM, 1)
dfci.sort_values('Edav')

,Group,Spin,Irrep,Label,Energy,Edav,Ncore,dipX,dipY,dipZ,Eref,Dipole,Ref,C0,Configs,Lz,Term,record,trig,ecm
0,1,Singlet,1,1.1,-119.804437,-119.902534,0,0.0,0.0,-0.013448,-119.075073,0.013448,1.1,0.939779,"{'2222ab22222': 0.366844, '22222022222': 0.857...",0.0,1Σ+,5101.2,1,0.0
34,17,Triplet,4,1.4,-119.782434,-119.863230,0,0.0,0.0,0.398409,-119.078953,0.398409,1.4,0.947100,"{'2222a22222a': 0.0631749, '22222a2222a': 0.94...",2.0,3Δ,6402.2,sin,8626.3
22,11,Triplet,1,1.1,-119.782434,-119.863230,0,0.0,0.0,0.398410,-119.078953,0.398410,1.1,0.947100,"{'222aa222222': 0.0631749, '222a2a22222': 0.94...",2.0,3Δ,6102.2,cos,8626.3
5,2,Singlet,1,2.1,-119.778303,-119.860163,0,0.0,0.0,0.402465,-119.072435,0.402465,2.1,0.946625,"{'222a2b22222': 0.9384721, '222ab222222': 0.10...",2.0,1Δ,5102.2,cos,9299.3
17,8,Singlet,4,1.4,-119.778303,-119.860163,0,0.0,0.0,0.402463,-119.072435,0.402463,1.4,0.946625,"{'22222a2222b': 0.9384721, '2222222a2b2': 0.05...",2.0,1Δ,5401.2,sin,9299.3
24,12,Triplet,2,1.2,-119.774487,-119.855430,0,0.0,0.0,0.347584,-119.070139,0.347584,1.2,0.947068,"{'22222a2a222': 0.9390938, '2222a22a222': 0.11...",1.0,3Π,6201.2,cos,10338.2
28,14,Triplet,3,1.3,-119.774487,-119.855430,0,0.0,0.0,0.347584,-119.070139,0.347584,1.3,0.947068,"{'22222a222a2': 0.9390938, '2222a2222a2': 0.11...",1.0,3Π,6301.2,sin,10338.2
21,10,Triplet,1,2.1,-119.773310,-119.854977,0,0.0,0.0,0.251031,-119.063229,0.251031,2.1,0.947023,"{'2222aa22222': 0.9394575, '22a22a22222': 0.10...",0.0,3Σ+,6101.2,1,10437.6
9,4,Singlet,2,1.2,-119.758579,-119.841284,0,0.0,0.0,0.391584,-119.049726,0.391584,1.2,0.946320,"{'22222a2b222': 0.9305114, '2222a22b222': 0.15...",1.0,1Π,5201.2,cos,13442.7
13,6,Singlet,3,1.3,-119.758579,-119.841284,0,0.0,0.0,0.391584,-119.049726,0.391584,1.3,0.946320,"{'22222a222b2': 0.9305114, '2222a2222b2': 0.15...",1.0,1Π,5301.2,sin,13442.7


In [13]:
# check for erroneously repeated CI roots
tol = 1.e-6
for irrep, grp in dfci.sort_values('Energy').groupby(['Spin', 'Irrep']):
    e = grp.Energy.values
    de = e[1:] - e[:-1]
    smal = np.abs(de) < tol
    for i, s in enumerate(smal):
        if s:
            print('Warning: closely repeated root in MRCI')
            display(grp.iloc[[i,i+1]])
print('Checking for discrepancies between reference energy and CASSCF energy')
dfcheck = dfci[['Spin', 'Irrep', 'Label', 'Edav', 'Eref']].copy()
ecas = []
for i, row in dfcheck.iterrows():
    ecas.append(CAS.results[(CAS.results.Label == row.Label) & (CAS.results.Spin == row.Spin)].Energy.values[0])
dfcheck['CAS'] = ecas
dfcheck['diff'] = np.round(dfcheck.CAS - dfcheck.Eref, 6)
dfbad = dfcheck[np.abs(dfcheck['diff']) > 0.1].sort_values(['Spin', 'Irrep', 'Label'])
if len(dfbad):
    display(dfbad)
else:
    print('\t--looks good')

Checking for discrepancies between reference energy and CASSCF energy
	--looks good


In [14]:
# read SO-CI
SOCI = mpr.fullmatSOCI(fsoci, hybrid=True)
dfterms = SOCI.average_terms()

Computational group = C2v
CASSCF states:
    21 Singlet
    15 Triplet
Replacing MRCI+Q energies by HLSDIAG values
*** Warning: "66 SO states but only 44 unique basis state labels" in parse_SOmatrix()


In [15]:
# Assign leading terms
lTerm = []  # list of str
for ilead in np.argmax(SOCI.termwt, axis=0):
    lTerm.append(SOCI.dfterm.at[ilead, 'Term'])
dfso = SOCI.SOe.results.copy()
dfso['Leading'] = lTerm
# All term weights
twts = []   # list of dict
for iso in range(SOCI.dimen):
    twts.append({t: np.round(w, 4) for t, w in zip(SOCI.dfterm.Term.values, SOCI.termwt[:, iso])})
dfso['Termwts'] = twts
print('Term weights rounded to 1e-4')
dfso = dfso.rename(columns={'Erel': 'cm-1'})  # rename column Erel to cm-1
SOCI.dfso = dfso
dfso

Term weights rounded to 1e-4


,Nr,Irrep,E,Eshift,cm-1,Leading,Termwts
0,1,1,-119.908022,-1204.51,0.00,(1)1Σ+,"{'(1)1Σ+': 0.9148, '(1)3Δ': 0.0, '(1)1Δ': 0.0,..."
1,37,3,-119.881322,4655.51,5860.02,(1)3Δ,"{'(1)1Σ+': 0.0, '(1)3Δ': 0.9966, '(1)1Δ': 0.0,..."
2,22,2,-119.881322,4655.51,5860.02,(1)3Δ,"{'(1)1Σ+': 0.0, '(1)3Δ': 0.9966, '(1)1Δ': 0.0,..."
3,52,4,-119.879937,4959.52,6164.03,(1)3Δ,"{'(1)1Σ+': 0.0, '(1)3Δ': 0.5995, '(1)1Δ': 0.37..."
4,2,1,-119.879935,4959.83,6164.34,(1)3Δ,"{'(1)1Σ+': 0.0, '(1)3Δ': 0.5992, '(1)1Δ': 0.37..."
5,3,1,-119.874833,6079.76,7284.27,(1)3Π,"{'(1)1Σ+': 0.0, '(1)3Δ': 0.0506, '(1)1Δ': 0.22..."
6,53,4,-119.874825,6081.38,7285.88,(1)3Π,"{'(1)1Σ+': 0.0, '(1)3Δ': 0.0503, '(1)1Δ': 0.22..."
7,38,3,-119.872496,6592.64,7797.14,(1)3Π,"{'(1)1Σ+': 0.0, '(1)3Δ': 0.242, '(1)1Δ': 0.0, ..."
8,23,2,-119.872496,6592.64,7797.14,(1)3Π,"{'(1)1Σ+': 0.0, '(1)3Δ': 0.242, '(1)1Δ': 0.0, ..."
9,54,4,-119.868586,7450.77,8655.28,3Σ+,"{'(1)1Σ+': 0.0, '(1)3Δ': 0.0, '(1)1Δ': 0.0, '(..."


In [16]:
# Create a DataFrame to describe the basis states
cilbl = []
irrep = []
Ss = []
Szs = []
for bas in SOCI.basis:
    cilbl.append(bas[0])
    i, irp = bas[0].split('.')
    irrep.append(irp)
    Ss.append(bas[1])
    Szs.append(bas[2])
dfbas = pd.DataFrame({'cilbl': cilbl, 'irrep': irrep, 'S': Ss, 'Sz': Szs})
# Add values of |Lz|
Lz = [dfci.loc[ici, 'Lz'] for ici in SOCI.sob_ici]
dfbas['|Lz|'] = Lz

In [17]:
# Pair the basis states that are sin/cos siblings
sib = [-1] * SOCI.dimen
trig = [dfci.at[ici, 'trig'] for ici in SOCI.sob_ici]
dfbas['trig'] = trig
baspairs = []
for iterm, trow in SOCI.dfterm.iterrows():
    ibass = SOCI.term_iso[iterm]
    for ibas in ibass:
        if sib[ibas] > -1:
            # already assigned
            continue
        for jbas in ibass:
            if (jbas != ibas) and (dfbas.at[ibas, 'S'] == dfbas.at[jbas, 'S']) and \
                            (dfbas.at[ibas, 'Sz'] == dfbas.at[jbas, 'Sz']):
                sib[ibas] = jbas
                sib[jbas] = ibas
                baspairs.append([ibas, jbas])
dfbas['sib'] = sib
print('Basis states')
dfbas

Basis states


,cilbl,irrep,S,Sz,|Lz|,trig,sib
0,1.1,1,0.0,0.0,0.0,1,-1
1,3.1,1,0.0,0.0,0.0,1,-1
2,4.1,1,0.0,0.0,0.0,1,-1
3,7.1,1,0.0,0.0,0.0,1,-1
4,9.1,1,0.0,0.0,0.0,1,-1
5,2.1,1,0.0,0.0,2.0,cos,17
6,5.1,1,0.0,0.0,2.0,cos,18
7,6.1,1,0.0,0.0,2.0,cos,19
8,8.1,1,0.0,0.0,4.0,cos,20
9,1.2,2,0.0,0.0,1.0,cos,13


In [18]:
# Begin assignments
omegavals = mpr.omega_counts(mrci, silent=True)
print('Target omega counts:', omegavals)
omeganeed = omegavals.copy()
# add column of possible Omegas to dfso
omposs = [set(omegavals.keys()) for i in range(SOCI.dimen)]
dfso['omposs'] = omposs
dfso[chem.OMEGA] = None
wthresh = 0.01  # ignore weights smaller than 'wthresh'
print('Weight threshold = {:.4f}'.format(wthresh))

Target omega counts: {0: 14, 1: 22, 2: 18, 3: 8, 4: 4}
Weight threshold = 0.0100


In [19]:
def assign_state(iso, omega, omeganeed, dfso):
    # Assign the state 'iso' to Ω = 'omega'
    #   'omeganeed' is the dict of needed values (modified)
    #   'dfso' is a DataFrame of SO states (modified)
    # Return success flag and a message

    # Is this state already assigned?
    if dfso.at[iso, 'Ω'] is not None:
        # already assigned
        ok = (omega == dfso.at[iso, 'Ω'])  # failure if this is a different value
        return ok, f'conflict of {omega} with earlier assignment {dfso.at[iso, "Ω"]}'
    # Is 'omega' among the remaining possiblities for this state?
    if not omega in dfso.at[iso, 'omposs']:
        return False, 'already excluded for this state'
    # Is 'omega' still needed?
    if omeganeed[omega] < 1:
        return False, f'exceeds target count for Ω = {omega}'
    # update 'omeganeed' and 'dfso'
    omeganeed[omega] -= 1
    dfso.at[iso, 'omposs'] = set([omega])
    dfso.at[iso, 'Ω'] = omega
    return True, 'new assignment'

def check_fulfillment(omeganeed, dfso, verbose=True):
    # If any omega counts have been reached, delete
    #   the corresponding omegas from possible future
    #   assignments
    # Return the number of affected states
    totchange = 0
    for om, n in omeganeed.items():
        nchange = 0
        if n > 1:
            # count is not satisfied 
            continue
        # count is satisfied for Omega = om
        for iso, row in dfso.iterrows():
            oset = row['omposs']
            if (len(oset) > 1) and (om in oset):
                # it is listed; remove it
                oset.remove(om)
                dfso.at[iso, 'omposs'] = oset
                nchange += 1
        if nchange and verbose:
            print(f'Omega count satisfied for Ω = {om}: possibilities updated for {nchange} states')
        totchange += nchange
    return totchange

def assign_singletons(dfso, omeganeed):
    # If there are unassigned states with only one possible
    #   value of Omega, assign them
    # Return the number of states so assigned
    inotassigned = dfso[dfso.Ω.isnull()].index
    nnew = 0
    for iso in inotassigned:
        omposs = list(dfso.at[iso, 'omposs'])
        nposs = len(omposs)
        if nposs == 1:
            # assign this state
            assign_state(iso, omposs[0], omeganeed, dfso)
            nnew += 1
    return nnew

In [20]:
# Sigma states are the easiest, so start with them
eigvec = SOCI.SOvec.copy()  # Eigenvectors from Molpro; will it work with my vectors?
eigvec = SOCI.vec  # eigenvectors from re-diagonalization
print('Starting with Lz = 0 basis states')
nnew = 0
for ibas, row in dfbas.iterrows():
    if row['|Lz|'] != 0:
        continue
    # Lz = 0
    om = abs(row.Sz)  # Ω = 0 + Sz
    #print(ibas, row['|Lz|'], om)
    # Assign the SO states to which this BS contributes sufficient weight
    wts = np.absolute(eigvec[ibas, :])
    idx = np.argwhere(wts > wthresh).flatten()
    for iso in idx:
        ok, msg = assign_state(iso, om, omeganeed, dfso)
        if ok and (msg == 'new assignment'):
            print(f'\tassigned state #{iso} with Ω = {om}')
            nnew += 1
        if not ok:
            print(f'*** Assignment failed:  ibas = {ibas}, iso = {iso}, omega = {om}: {msg}')
print(f'\t---{nnew} states assigned---')
print('Omegas still needed: ', omeganeed)

Starting with Lz = 0 basis states
	assigned state #0 with Ω = 0.0
	assigned state #12 with Ω = 0.0
	assigned state #18 with Ω = 0.0
	assigned state #27 with Ω = 0.0
	assigned state #46 with Ω = 0.0
	assigned state #30 with Ω = 0.0
	assigned state #49 with Ω = 0.0
	assigned state #60 with Ω = 0.0
	assigned state #47 with Ω = 0.0
	assigned state #65 with Ω = 0.0
	assigned state #7 with Ω = 1.0
	assigned state #8 with Ω = 1.0
	assigned state #10 with Ω = 1.0
	assigned state #11 with Ω = 1.0
	assigned state #13 with Ω = 1.0
	assigned state #14 with Ω = 1.0
	assigned state #19 with Ω = 1.0
	assigned state #20 with Ω = 1.0
	assigned state #25 with Ω = 1.0
	assigned state #26 with Ω = 1.0
	assigned state #31 with Ω = 1.0
	assigned state #32 with Ω = 1.0
	assigned state #36 with Ω = 1.0
	assigned state #37 with Ω = 1.0
	assigned state #42 with Ω = 1.0
	assigned state #43 with Ω = 1.0
	assigned state #50 with Ω = 1.0
	assigned state #51 with Ω = 1.0
	assigned state #9 with Ω = 0.0
	assigned sta

In [21]:
npruned = check_fulfillment(omeganeed, dfso)
while npruned:
    # assign any states with only one remaining possibility
    npruned = 0
    nnew = assign_singletons(dfso, omeganeed)
    if nnew:
        npruned = check_fulfillment(omeganeed, dfso)
print('Assignments still needed:', omeganeed)

Omega count satisfied for Ω = 0: possibilities updated for 30 states
Omega count satisfied for Ω = 1: possibilities updated for 30 states
Assignments still needed: {0: 0, 1: 0, 2: 18, 3: 8, 4: 4}


In [22]:
# The trig method does not include basis states with Lz = 0
print('Using trig method to assign more states')
print('Everything below is numbered from zero, but in Molpro is numbered from 1')
Omegas = [None] * SOCI.dimen
for iso in range(SOCI.dimen):
    print(f'Eigenvector #{iso}')
    vec = eigvec[:, iso]
    #print(np.round(vec, 6))
    omset = set()  # set of found Omega values--should end up with only one element
    usedBS = []  # list of basis states considered
    # Basis states are considered in pairs
    for baspair in baspairs:
        # 'a' and 'b' are coeffs of cos() and sin()
        [i, j] = baspair
        usedBS.extend(baspair)
        aLz = dfbas.at[i, '|Lz|']  # same for basis-state[i] and BS[j]
        Sz = dfbas.at[i, 'Sz']  # same for i and j
        if dfbas.at[i, 'trig'] == 'cos':
            # cos, sin ordering
            [a, b] = [vec[i], vec[j]]
        else:
            # sin, cos ordering
            [a, b] = [vec[j], vec[i]]
        #print('coeffs:', vec[i], vec[j])
        alen = np.absolute(a)
        blen = np.absolute(b)
        magn = (alen + blen) / 2
        delta = alen - blen
        if abs(delta) > wthresh:
            print('\tBSs:', baspair, f'with magn = {magn:.3f}')
            print('** large difference in magnitudes = {:.8f}'.format(delta))
            # do not include this pair
            continue
        if min(alen, blen) < wthresh:
            # ignore small coefficients
            continue
        # divide by a to get standard form
        b /= a
        a = 1
        #print('a, b  :', a, b)
        dev = abs(np.imag(b)) - 1
        if abs(dev) > wthresh:
            print('\tBSs:', baspair, f'with magn = {magn:.3f}')
            print('** imag(b) deviates from unity by: {:.8f}'.format(dev))
        if np.imag(b) > wthresh:
            # a positive combination
            Lz = aLz
        elif np.imag(b) < -wthresh:
            # a negative combination
            Lz = -aLz
        Omz = Lz + Sz
        print('\tBSs:', baspair, f'with magn = {magn:.3f}')
        print(f'\t\tLz = {Lz}, Sz = {Sz}, Omz = {Omz}')
        omset = omset.union([abs(Omz)])
        
    # finish up
    Omegas[iso] = omset
    if len(omset) == 1:
        Omegas[iso] = omset
    elif len(omset) > 1:
        print('\t*** Multiple Omega values:', omset)
    else:
        print('\t*** No remaining Omega values')

Using trig method to assign more states
Everything below is numbered from zero, but in Molpro is numbered from 1
Eigenvector #0
	BSs: [30, 42] with magn = 0.144
		Lz = -1.0, Sz = 1.0, Omz = 0.0
	BSs: [36, 48] with magn = 0.144
		Lz = 1.0, Sz = -1.0, Omz = 0.0
	BSs: [31, 43] with magn = 0.010
		Lz = -1.0, Sz = 1.0, Omz = 0.0
	BSs: [37, 49] with magn = 0.010
		Lz = 1.0, Sz = -1.0, Omz = 0.0
Eigenvector #1
	BSs: [60, 24] with magn = 0.448
		Lz = -2.0, Sz = 1.0, Omz = -1.0
	BSs: [64, 28] with magn = 0.545
		Lz = 2.0, Sz = -1.0, Omz = 1.0
	BSs: [40, 52] with magn = 0.026
** large difference in magnitudes = -0.04203794
	BSs: [12, 16] with magn = 0.018
** large difference in magnitudes = 0.02996877
Eigenvector #2
	BSs: [60, 24] with magn = 0.545
		Lz = -2.0, Sz = 1.0, Omz = -1.0
	BSs: [64, 28] with magn = 0.448
		Lz = 2.0, Sz = -1.0, Omz = 1.0
	BSs: [40, 52] with magn = 0.026
** large difference in magnitudes = 0.04203794
	BSs: [12, 16] with magn = 0.018
** large difference in magnitudes = -0

	BSs: [9, 13] with magn = 0.007
** large difference in magnitudes = 0.01466871
	BSs: [61, 25] with magn = 0.118
		Lz = 2.0, Sz = 1.0, Omz = 3.0
	BSs: [65, 29] with magn = 0.118
		Lz = -2.0, Sz = -1.0, Omz = -3.0
	BSs: [34, 46] with magn = 0.156
** large difference in magnitudes = -0.31109393
	BSs: [14, 10] with magn = 0.334
** large difference in magnitudes = 0.66770746
	BSs: [35, 47] with magn = 0.241
** large difference in magnitudes = -0.48264334
	BSs: [11, 15] with magn = 0.119
** large difference in magnitudes = 0.23845927
Eigenvector #33
	BSs: [63, 27] with magn = 0.280
** large difference in magnitudes = -0.56000275
	BSs: [31, 43] with magn = 0.307
		Lz = 1.0, Sz = 1.0, Omz = 2.0
	BSs: [37, 49] with magn = 0.307
		Lz = -1.0, Sz = -1.0, Omz = -2.0
	BSs: [32, 44] with magn = 0.194
		Lz = 1.0, Sz = 1.0, Omz = 2.0
	BSs: [38, 50] with magn = 0.194
		Lz = -1.0, Sz = -1.0, Omz = -2.0
	BSs: [18, 6] with magn = 0.132
** large difference in magnitudes = 0.26324169
	BSs: [7, 19] with magn 

In [23]:
# Assign states where trig analysis gave a single Omega
print('Assigning states with clean trig analysis')
nnew = 0
for iso, omset in enumerate(Omegas):
    if len(omset) == 1:
        # a clean value
        om = list(omset)[0]
        ok, msg = assign_state(iso, om, omeganeed, dfso)
        if ok and (msg == 'new assignment'):
            print(f'\tassigned state #{iso} with Ω = {om}')
            nnew += 1
        if not ok:
            print(f'*** Assignment failed: iso = {iso}, omega = {om}: {msg}')
print(f'\t---{nnew} states assigned---')
print('Omegas still needed: ', omeganeed)

Assigning states with clean trig analysis
*** Assignment failed: iso = 1, omega = 1.0: already excluded for this state
*** Assignment failed: iso = 2, omega = 1.0: already excluded for this state
	assigned state #3 with Ω = 2.0
	assigned state #4 with Ω = 2.0
	assigned state #5 with Ω = 2.0
	assigned state #6 with Ω = 2.0
*** Assignment failed: iso = 7, omega = 3.0: conflict of 3.0 with earlier assignment 1.0
*** Assignment failed: iso = 8, omega = 3.0: conflict of 3.0 with earlier assignment 1.0
*** Assignment failed: iso = 10, omega = 3.0: conflict of 3.0 with earlier assignment 1.0
*** Assignment failed: iso = 11, omega = 3.0: conflict of 3.0 with earlier assignment 1.0
*** Assignment failed: iso = 13, omega = 3.0: conflict of 3.0 with earlier assignment 1.0
*** Assignment failed: iso = 14, omega = 3.0: conflict of 3.0 with earlier assignment 1.0
	assigned state #15 with Ω = 2.0
	assigned state #16 with Ω = 2.0
*** Assignment failed: iso = 19, omega = 3.0: conflict of 3.0 with earli

In [24]:
# Assign states where trig analysis gave multiple Omegas
print('Assigning states with mixed trig analysis')
nnew = 0
for iso, row in dfso.iterrows():
    if len(Omegas[iso]) < 2:
        continue
    oms = row.omposs.intersection(Omegas[iso])  # intersection of omposs with trig results
    if len(oms) == 1:
        # a possible new assignment
        om = list(oms)[0]
        ok, msg = assign_state(iso, om, omeganeed, dfso)
        if ok and (msg == 'new assignment'):
            print(f'\tassigned state #{iso} with Ω = {om}')
            nnew += 1
        if not ok:
            print(f'*** Assignment failed:  ibas = {ibas}, iso = {iso}, omega = {om}: {msg}')
print(f'\t---{nnew} states assigned---')
print('Omegas still needed: ', omeganeed)

Assigning states with mixed trig analysis
	---0 states assigned---
Omegas still needed:  {0: 0, 1: 0, 2: 0, 3: 8, 4: 0}


In [25]:
npruned = check_fulfillment(omeganeed, dfso)
while npruned:
    # assign any states with only one remaining possilibity
    npruned = 0
    nnew = assign_singletons(dfso, omeganeed)
    if nnew:
        npruned = check_fulfillment(omeganeed, dfso)
nneed = sum(omeganeed.values())
if nneed:
    print('Assignments still needed:', omeganeed)
else:
    print('All states assigned!')

Omega count satisfied for Ω = 2: possibilities updated for 8 states
Omega count satisfied for Ω = 4: possibilities updated for 8 states
All states assigned!


In [26]:
# Add term symbol labels and Omega labels
tsymb = []
olbl = []
#for trm, om in zip(dfso.Leading, dfso.Ω):
for irow, row in dfso.iterrows():
    om = row.Ω
    hom = chem.halves(om)
    if hom == '0':
        # assign parity based upon irrep
        irr = row.Irrep 
        if irr == 1:
            hom += '+'
        elif irr == 4:
            hom += '-'
        else:
            # should not happen
            hom += f':{irr}'
    olbl.append(hom)
    trm = row.Leading
    tsymb.append('_'.join([trm, hom]))
dfso['Tsymb'] = tsymb
dfso['Olbl'] = chem.enumerative_prefix(olbl, always=True)
SOCI.average_SO_levels(be_same=['Ω'], to_avg=['E', 'Eshift', 'cm-1'])
dflevel = SOCI.dflevel.reset_index(drop=True)
print(f'Assignments for {fsoci}')
dflevel[['Nr', 'E', 'Eshift', 'cm-1', 'Leading', 'Ω', 'g', 'Tsymb', 'Olbl']]

Difference in E of 1.9e-05 exceeds desired tolerance of 1.0e-05 for this pair:


,Nr,Irrep,E,Eshift,cm-1,Leading,Termwts,omposs,Ω,Tsymb,Olbl,g
44,62,4,-119.714962,41167.26,42371.76,3Φ,"{'(1)1Σ+': 0.0, '(1)3Δ': 0.0026, '(1)1Δ': 0.00...",{2.0},2.0,3Φ_2,(13)2,2
45,13,1,-119.714943,41171.52,42376.03,3Φ,"{'(1)1Σ+': 0.0, '(1)3Δ': 0.0026, '(1)1Δ': 0.00...",{2.0},2.0,3Φ_2,(14)2,2


Assignments for ac5z_soc2011_r1p5987_21S15T_lz.pro


,Nr,E,Eshift,cm-1,Leading,Ω,g,Tsymb,Olbl
0,[1],-119.908022,-1204.510,0.000,(1)1Σ+,0.0,1,(1)1Σ+_0+,(1)0+
1,"[37, 22]",-119.881322,4655.510,5860.020,(1)3Δ,3,2,(1)3Δ_3,(1)3
2,"[52, 2]",-119.879936,4959.675,6164.185,(1)3Δ,2.0,2,(1)3Δ_2,(1)2
3,"[3, 53]",-119.874829,6080.570,7285.075,(1)3Π,2.0,2,(1)3Π_2,(3)2
4,"[38, 23]",-119.872496,6592.640,7797.140,(1)3Π,1.0,2,(1)3Π_1,(1)1
5,[54],-119.868586,7450.770,8655.280,3Σ+,0.0,1,3Σ+_0-,(1)0-
6,"[39, 24]",-119.864925,8254.160,9458.670,3Σ+,1.0,2,3Σ+_1,(3)1
7,[4],-119.848694,11816.490,13021.000,(1)3Π,0.0,1,(1)3Π_0+,(2)0+
8,"[25, 40]",-119.834937,14835.930,16040.440,(1)3Δ,1.0,2,(1)3Δ_1,(5)1
9,"[55, 5]",-119.834246,14987.395,16191.900,(1)1Δ,2.0,2,(1)1Δ_2,(5)2


In [27]:
# Is this a hybrid calculation (prepared by build_hybrid_soci_input.ipynb)?
ccterms = []  # list of input CCSD(T) terms
rx_hyb = re.compile('HLSDIAG\(.+\s+!\s*.*(ccsd|anchored|shifted|input)')
is_hybrid = False
with open(fsoci, 'r', encoding='utf8') as F:
    for line in F:
        if rx_hyb.search(line):
            is_hybrid = True
            if 'input' in line:
                # an anchor term; extract its label
                words = line.split()
                if words[2] not in ccterms:
                    ccterms.append(words[2])
if is_hybrid:
    print('Hybrid SO-CI calculation')
else:
    print('Standard SO-CI')

Standard SO-CI


In [28]:
# copy Energies to clipboard with term symbols as column headings
dfcp = dflevel[['Olbl', 'E']].copy().sort_values('Olbl').set_index('Olbl')
dfcp.rename(columns={'E': f'{R}'}, inplace=True)   # put bond length in that position for pasting to Excel
dfcp.T.to_clipboard()
if is_hybrid:
    print(f'Hybrid SO-CI energies for R={R} copied to clipboard, for pasting into Excel')
else:
    print(f'Standard SO-CI energies for R={R} copied to clipboard, for pasting into Excel')

Standard SO-CI energies for R=1.5987 copied to clipboard, for pasting into Excel


### Term composition of some levels

In [ ]:
olabels = ['(1)1/2', '(1)3/2', '(1)5/2']
for olabel in olabels:
    ilev = dfso[SOCI.dfso.Olbl == olabel].index[0]
    #print('ilvel =', ilev)
    print('Composition of level #{:d}:  "{:s}" or "{:s}"'.format(ilev, dfso.loc[ilev].Olbl,
                                                             dfso.loc[ilev].Tsymb))
    dfci, dfterm = SOCI.composition_of_level(ilev, thr=1.e-6, normalize=True)
    wtcol = dfterm.columns[-1]
    display(dfterm.sort_values(wtcol, ascending=False))
    print('Sum of weights = {:.3f}'.format(dfterm[wtcol].sum()))
    ccsum = dfterm[dfterm.Term.isin(ccterms)][wtcol].sum()
    print('Sum of weights from CC terms = {:.3f}\n'.format(ccsum))

### Distribution of a term among levels

In [ ]:
term = '(1)2Σ+'
term = '(1)2Δ'
if is_hybrid:
    typ = 'hybrid'
else:
    typ = 'standard'
print('Distribution of term "{:s}" among {:s} levels'.format(term, typ))
print(f'R = {R}')
df = SOCI.level_contributions_from_term(term, thr=1.e-6, normalize=True,
                                       cols=['E', 'Eshift', 'Ω', 'Tsymb', 'Olbl'])
df['prod'] = np.round(df.Eshift * df[term], 1)
ebar = df['prod'].sum() / df[term].sum()
print('Weighted mean energy of {:s} = {:.1f} cm-1'.format(term, ebar))
#display(df.sort_values(term, ascending=False))
fmt = {term: '{:.4f}', 'prod': '{:.2f}', 'Eshift': '{:.2f}', 'Ω': '{:.1f}'}
display(df[df[term] > 0.0005].sort_values('E').style.format(fmt))
print('Total weight = {:.4f}'.format(df[term].sum()))

### Contribution of CCSD(T) terms to all levels (hybrid SO-CI)

In [ ]:
# Get contributions of CC terms to all levels
# Loop over levels to get the normalization right
dfcc = SOCI.dfso.copy()
ccsums = []
for ilev in dfcc.index:
    dfci, dfterm = SOCI.composition_of_level(ilev, thr=1.e-6,
                                normalize=True, silent=True)
    wtcol = dfterm.columns[-1]
    ccsum = dfterm[dfterm.Term.isin(ccterms)][wtcol].sum()
    ccsums.append(ccsum)
dfcc['CCwt'] = ccsums
fmt = {chem.OMEGA: '{:.1f}', 'exc': '{:.2f}', 'CCwt': '{:.3f}'}
display(dfcc[dfcc.columns[2:]].style.format(fmt))
print('Total of CCwt column = {:.3f}'.format(dfcc.CCwt.sum()))
print('The CC terms are', ccterms)